In [15]:
import chess
import pandas as pd
from collections import defaultdict as dd
import numpy as np
import chess.engine
from sklearn.preprocessing import OneHotEncoder
engine = chess.engine.SimpleEngine.popen_uci(r"C:\Users\prvn0\Documents\Chess\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe")

In [16]:
def clean(x):
    board = chess.Board(fen=x["FEN"])
    x["Moves"] = [chess.Move.from_uci(move) for move in x["Moves"].split()]
    board.push(x["Moves"].pop(0))
    x["FEN"] = board.fen()
    return x

dataset = pd.read_csv(r"C:\Users\prvn0\Documents\Chess\lichess_db_puzzle.csv")
samples = dataset[:200].apply(lambda x:clean(x), axis=1)


In [17]:
piece_values = {
    chess.PAWN: 1,
    chess.KNIGHT: 3,
    chess.BISHOP: 3,
    chess.ROOK: 5,
    chess.QUEEN: 9, 
    chess.KING: 0
}

In [18]:
# one hot encode Themes
themes=[]
for theme_list in dataset['Themes']:
    for word in theme_list.split():
        themes.append(word)
themes = np.unique(themes)
encoder = OneHotEncoder()
encoder.fit_transform(themes.reshape(-1, 1))


<60x60 sparse matrix of type '<class 'numpy.float64'>'
	with 60 stored elements in Compressed Sparse Row format>

In [19]:
def get_material_difference(board, my_color):
    return sum(piece_values[piece.piece_type] for piece in board.piece_map().values() if piece.color == my_color) - sum(piece_values[piece.piece_type] for piece in board.piece_map().values() if piece.color != my_color)
X={"material_sacrifice": [], "puzzle_length": [], "Themes":[], "num_captured": [], "num_being_captured": [], "material_difference": [], "bad_checks": [], "bad_captures": []}
y=samples["Rating"]
for _, puzzle in samples.iterrows():
    board = chess.Board(puzzle["FEN"])
    my_color = board.turn
    captured = 0
    being_captured = 0
    total_sacrifice = 0
    exchange_material_loss = 0
    bad_checks = 0
    bad_captures = 0
    for move in puzzle["Moves"]:
        if board.is_capture(move):
            captured_piece = board.piece_at(move.to_square)
            if captured_piece.color == my_color:
                exchange_material_loss += piece_values[captured_piece.piece_type]
                being_captured += 1
            else:
                exchange_material_loss -= piece_values[captured_piece.piece_type]
                captured += 1
        elif exchange_material_loss:
            # the capture series end and someone lost material
            if exchange_material_loss > 0:
                #if I lost more materials for win/advantage
                total_sacrifice += exchange_material_loss

            exchange_material_loss = 0

        # check tempting bad moves
        if board.turn == my_color:

            bad_moves = list(board.legal_moves)
            bad_moves.remove(move)
            for bad_move in bad_moves:
                if board.is_capture(bad_move):
                    bad_captures += 1
                    
                if board.gives_check(bad_move):
                    bad_checks += 1

                    
        
        #explore next move
        board.push(move)

    if exchange_material_loss > 0:
        total_sacrifice += exchange_material_loss
        
    X["material_sacrifice"].append(total_sacrifice)
    X["puzzle_length"].append(len(puzzle["Moves"]))
    X["Themes"].append(encoder.transform(np.array(puzzle["Themes"].split()).reshape(-1, 1)).toarray())
    X["num_captured"].append(captured)
    X["num_being_captured"].append(being_captured)
    X["material_difference"].append(get_material_difference(board, my_color))
    X["bad_checks"] = bad_checks
    X["bad_captures"] = bad_captures

In [27]:
df = pd.DataFrame(X)
df[:10]

,material_sacrifice,puzzle_length,Themes,num_captured,num_being_captured,material_difference,bad_checks,bad_captures
0,0,5,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2,1,6,0,0
1,0,3,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1,1,4,0,0
2,0,3,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1,0,0,0,0
3,0,3,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2,1,2,0,0
4,0,5,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",3,0,2,0,0
5,0,3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0,0,-4,0,0
6,0,3,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2,1,3,0,0
7,0,3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1,0,-2,0,0
8,1,3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",2,1,-2,0,0
9,0,5,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1,0,3,0,0


In [ ]:
k=122
print(y[k])
X["puzzle_length"][k]

1223


3

In [ ]:
samples.iloc[k]

PuzzleId                                                      007eS
FEN                6k1/p4p2/1p5p/4r3/P3B3/1P2KP2/2P3PP/8 b - - 1 29
Moves                                            [f7f5, g2g4, f5e4]
Rating                                                         1223
RatingDeviation                                                  74
Popularity                                                       97
NbPlays                                                        8936
Themes                                      advantage endgame short
GameUrl                             https://lichess.org/8qs8bafy#56
OpeningTags                                                     NaN
Name: 122, dtype: object

In [ ]:
for k in (116,):
    info = engine.analyse(chess.Board(samples.iloc[k]["FEN"]), chess.engine.Limit(time=0.01))
    print(info["score"].wdl())

PovWdl(Wdl(wins=1000, draws=0, losses=0), WHITE)


In [ ]:
move = chess.Move.from_uci("f1g2")
bad_moves = list(board.legal_moves)
bad_moves.remove(move)
print(bad_moves)


ValueError: list.remove(x): x not in list

In [ ]:
type(board.legal_moves)